In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import entropy
import numpy as np

def stratified_random_sample(X, y, fraction=None, count=None, random_state=42):
    assert (fraction is not None) ^ (count is not None), "Specify either fraction or count"

    rng = np.random.RandomState(seed=random_state)
    X_out, y_out = [], []

    for label in np.unique(y):
        X_c = X[y == label]
        y_c = y[y == label]

        n_total = len(X_c)
        n_sample = int(n_total * fraction) if fraction is not None else min(count, n_total)
        if n_sample == 0:
            n_sample = 1

        idx = rng.choice(n_total, size=n_sample, replace=False)

        X_out.append(X_c[idx])
        y_out.append(y_c[idx])

    return np.concatenate(X_out, axis=0), np.concatenate(y_out, axis=0)


def classwise_uncertainty_sampling_by_fraction(X_pool, y_pool, model, fraction=0.2, random_state=42):
    X_selected = []
    y_selected = []
    selected_indices = []

    classes = np.unique(y_pool)
    probs = model.predict_proba(X_pool)
    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        idx_c = np.where(y_pool == c)[0]
        X_c = X_pool[idx_c]
        probs_c = probs[idx_c]


        n_c = len(idx_c)
        n_select = max(1, int(n_c * fraction))

        if n_c == 0:
            continue

        uncertainties = entropy(probs_c.T)  # shape: (n_samples_in_class,)
        sorted_idx = np.argsort(uncertainties)[-n_select:]  # top-k uncertain

        top_k_idx = idx_c[sorted_idx]

        selected_indices.extend(top_k_idx)
        X_selected.append(X_pool[top_k_idx])
        y_selected.append(y_pool[top_k_idx])

    selected_indices = np.array(selected_indices)
    X_selected = np.concatenate(X_selected, axis=0)
    y_selected = np.concatenate(y_selected, axis=0)

    remaining_idx = np.setdiff1d(np.arange(len(X_pool)), selected_indices)
    X_rest = X_pool[remaining_idx]
    y_rest = y_pool[remaining_idx]

    return X_rest, X_selected, y_rest, y_selected

X_pool = x_test_adv_0_1 
y_pool = y_label_0_1
X_train_init, y_train_init = stratified_random_sample(X_pool, y_pool, fraction=0.1, random_state=42)

clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_init, y_train_init)



_, X_10, _, y_10 = classwise_uncertainty_sampling_by_fraction(
    x_test_adv_0_1, y_label_0_1, clf, fraction=0.1
)



In [6]:
X_10.shape

(205144, 56)

In [7]:
# from xgboost import XGBClassifier

# print('xgb')
# xgb10 = XGBClassifier()
# xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF
DT


DecisionTreeClassifier()

In [ ]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:


# base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = xgb10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_ActiveLearning/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [97603    80   695   446    52  3039     1   183 11703   116    53]
(97603, 56) (97603,)
Save 0 to baseline_Def1.npy
(80, 56) (80,)
Save 1 to baseline_Def2.npy
(695, 56) (695,)
Save 2 to baseline_Def3.npy
(446, 56) (446,)
Save 3 to baseline_Def4.npy
(52, 56) (52,)
Save 4 to baseline_Def5.npy
(3039, 56) (3039,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(183, 56) (183,)
Save 7 to baseline_Def8.npy
(11703, 56) (11703,)
Save 8 to baseline_Def9.npy
(116, 56) (116,)
Save 9 to baseline_Def10.npy
(53, 56) (53,)
Save 10 to baseline_Def11.npy
Execution Time: 0.245608 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [260296      7    645   1599     36  41606      1     65  37646     10
      2]
(260296, 56) (260296,)
Save 0 to BIM_Def1.npy
(7, 56) (7,)
Save 1 to BIM_Def2.npy
(645, 56) (645,)
Save 2 to BIM_Def3.npy
(1599, 56) (1599,)
Save 3 to BIM_Def4.npy
(36, 56) (36,)
Save 4 to BIM_Def5.npy
(41606, 56) (41606,)
Save 5 t

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ActiveLearning/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [96272    64   614   739    84  3878   231 11975    70    44]
(96272, 56) (96272,)
Save 0 to baseline_Def1.npy
(64, 56) (64,)
Save 1 to baseline_Def2.npy
(614, 56) (614,)
Save 2 to baseline_Def3.npy
(739, 56) (739,)
Save 3 to baseline_Def4.npy
(84, 56) (84,)
Save 4 to baseline_Def5.npy
(3878, 56) (3878,)
Save 5 to baseline_Def6.npy
(231, 56) (231,)
Save 7 to baseline_Def8.npy
(11975, 56) (11975,)
Save 8 to baseline_Def9.npy
(70, 56) (70,)
Save 9 to baseline_Def10.npy
(44, 56) (44,)
Save 10 to baseline_Def11.npy
Execution Time: 1.679992 seconds
(341913,)
[0 2 3 4 5 7 8 9] [241315    577    714      6  71374     13  27899     15]
(241315, 56) (241315,)
Save 0 to BIM_Def1.npy
(577, 56) (577,)
Save 2 to BIM_Def3.npy
(714, 56) (714,)
Save 3 to BIM_Def4.npy
(6, 56) (6,)
Save 4 to BIM_Def5.npy
(71374, 56) (71374,)
Save 5 to BIM_Def6.npy
(13, 56) (13,)
Save 7 to BIM_Def8.npy
(27899, 56) (27899,)
Save 8 to BIM_Def9.npy
(15, 56) (15,)
Save 9 to BIM_Def10

In [11]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_ActiveLearning/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [96996   157   838   635   147  5004     1   235  9898    27    33]
(96996, 56) (96996,)
Save 0 to baseline_Def1.npy
(157, 56) (157,)
Save 1 to baseline_Def2.npy
(838, 56) (838,)
Save 2 to baseline_Def3.npy
(635, 56) (635,)
Save 3 to baseline_Def4.npy
(147, 56) (147,)
Save 4 to baseline_Def5.npy
(5004, 56) (5004,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(235, 56) (235,)
Save 7 to baseline_Def8.npy
(9898, 56) (9898,)
Save 8 to baseline_Def9.npy
(27, 56) (27,)
Save 9 to baseline_Def10.npy
(33, 56) (33,)
Save 10 to baseline_Def11.npy
Execution Time: 0.128719 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [200398   2046   2058   3042    967 104459   1234   3218  24404     65
     22]
(200398, 56) (200398,)
Save 0 to BIM_Def1.npy
(2046, 56) (2046,)
Save 1 to BIM_Def2.npy
(2058, 56) (2058,)
Save 2 to BIM_Def3.npy
(3042, 56) (3042,)
Save 3 to BIM_Def4.npy
(967, 56) (967,)
Save 4 to BIM_Def5.npy
(104459, 56) (10445

In [12]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_ActiveLearning/UNSW_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [13]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_ActiveLearning.csv")

In [14]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,97603,80,695,446,52,3039,1,183,11703,116,53,113971
1,XGB20,260296,7,645,1599,36,41606,1,65,37646,10,2,341913
2,XGB20,215602,5,679,1622,35,85842,1,367,37748,10,2,341913
3,XGB20,260296,7,645,1599,36,41606,1,65,37646,10,2,341913
4,XGB20,125372,298,223,236,33,168009,42342,3235,2117,18,30,341913
5,XGB20,276528,228,1718,1210,142,43764,1,525,17381,260,156,341913
6,XGB20,286368,237,2231,1681,148,9436,4,544,40762,343,159,341913
7,XGB20,312831,24,52,1119,49,16656,2,96,10638,356,90,341913
8,XGB20,273104,12,53,1405,25,29464,1,8,37839,0,2,341913
9,XGB20,275481,8,638,1561,32,26253,1,32,37891,10,6,341913
